In [ ]:
from http.server import BaseHTTPRequestHandler, HTTPServer
import os
import json

#You should not change this lines.
#They define the parameters required to run the server.
hostName = "localhost"
serverPort = 8080


#MILESTONE 1
#endpoint => http://localhost:8080/test_connectivity_M1

#You just need to test the connectivity that returns the following JSON
#{"results": [{"Connectivity": "OK"}]}

#JSON TO DISPLAY IN CLIENT:
#{"results": [{"Connectivity": "OK"}]}

def test_connectivity_M1(text):
    results = {}
    json_result={}
    array = []             
    json_result['Connectivity']="OK"
    array.append(json_result)
    results['results'] = array
    api_answer=json.dumps(results, ensure_ascii=False)
    output_code=200
    return api_answer,output_code

#MILESTONE 2
#endpoint => http://localhost:8080/count_countries_M2
#Country Count. Provide the number of entries per country 
#in the dataset in a sorted way. First the most frequent country.
#TEXT TO DISPLAY IN CLIENT:
# 1: TR(317)
# 2: DE(317)
# 3: NO(315)
# 4: BR(312)
# 5: IR(311)
# 6: CA(297)
# 7: FR(297)
# 8: CH(296)
# 9: IE(293)
# 10: DK(290)
# 11: NZ(288)
# 12: AU(285)
# 13: ES(281)
# 14: GB(278)
# 15: NL(276)
# 16: FI(276)
# 17: US(271)

from collections import Counter

def country_count_M2(text):
    # Open and load the JSON data from the file
    with open('json_rand_users_5000.json', 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Extract the 'nat' field from each user
    nationalities = [user['nat'] for user in data['results'] if 'nat' in user]

    # Count occurrences of each nationality
    nationality_count = Counter(nationalities)

    # Sort the nationalities by count in descending order
    sorted_nationalities = sorted(nationality_count.items(), key=lambda x: x[1], reverse=True)

    # Prepare the response in the requested format
    json_result = {}
    for idx, (nat, count) in enumerate(sorted_nationalities, start=1):
        json_result[str(idx)] = f"{nat}({count})"

    # Prepare the final JSON response
    results = {"results": [json_result]}

    # Convert the results into JSON format
    api_answer = json.dumps(results, ensure_ascii=False)

    # Return the JSON response and the status code
    output_code = 200
    return api_answer, output_code
    
#MILESTONE 3    
#endpoint=>http://localhost:8080/count_countries_M3?countries=DK,ES,GB,IE,IR,NO,NL,NZ

#It is the same than before but in case it just returns the result for the provided countries.
#The list of available countries in the raduser API (field nat) is:
#AU, BR, CA, CH, DE, DK, ES, FI, FR, GB, IE, IR, NO, NL, NZ, TR, US
#If not countries parameter is provided you have to provide the results for all the countries
#NOTE: NO NEED TO IMPLEMENT CONTROL OF ERRORS

#TEXT TO DISPLAY IN CLIENT THAT ONLY CONSIDERS THE COUNTRIES IN THE QUERY:
# 1: NO(315)
# 2: IR(311)
# 3: IE(293)
# 4: DK(290)
# 5: NZ(288)
# 6: ES(281)
# 7: GB(278)
# 8: NL(276)

def country_count_M3(text):
    # Open and load the JSON data from the file
    with open('json_rand_users_5000.json', 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Extract the query string from the input text
    query_string = text.split('?', 1)[-1]  # Get everything after the '?'
    
    # Manually parse the query string for 'countries'
    countries_list = []
    if 'countries=' in query_string:
        # Get the part after 'countries='
        countries_param = query_string.split('countries=', 1)[1]
        # Split by comma and strip any whitespace
        countries_list = [country.strip() for country in countries_param.split(',')]
    
    # If no countries are provided, return an error response
    if not countries_list:
        return json.dumps({"error": "No countries specified"}), 400

    # Extract the 'nat' field from each user
    nationalities = [user['nat'] for user in data['results'] if 'nat' in user]

    # Count occurrences of each nationality
    nationality_count = Counter(nationalities)

    # Sort the nationalities by count in descending order
    sorted_nationalities = sorted(nationality_count.items(), key=lambda x: x[1], reverse=True)

    json_result = {}
    idx = 1
    for _, (nat, count) in enumerate(sorted_nationalities, start=1):
        if nat in countries_list:
            json_result[str(idx)] = f"{nat}({count})"
            idx += 1

    # Prepare the final JSON response
    results = {"results": [json_result]}

    # Convert the results into JSON format
    api_answer = json.dumps(results, ensure_ascii=False)

    # Return the JSON response and the status code
    output_code = 200
    return api_answer, output_code

#MILESTONE 4    
#endpoint=>http://localhost:8080/age_groups_M4 
#You need to deliver the number of users in the data set
#within the following age groups: <18, 18-29, 30-49, 50-64, >64.

#ANSWER IN CLIENT
# Age <18: 0
# Age 18-29: 634
# Age 30-49: 1762
# Age 50-64: 1452
# Age >64: 1152

def age_groups_M4(text):
    
    with open('json_rand_users_5000.json', 'r', encoding='utf-8') as file:
        data = json.load(file)

    # We will store the results in a ditionary so we initialize it with the groups ana all 0s
    age_groups = {
        "<18": 0,
        "18-29": 0,
        "30-49": 0,
        "50-64": 0,
        ">64": 0
    }

    # After taking a look in the provided document, we have seen that the age is inside the dob part, so we first
    # enter it and then we get the age and add one to the range it is within
    for user in data['results']:
        age = user['dob']['age']
        if age < 18:
            age_groups["<18"] += 1
        elif 18 <= age <= 29:
            age_groups["18-29"] += 1
        elif 30 <= age <= 49:
            age_groups["30-49"] += 1
        elif 50 <= age <= 64:
            age_groups["50-64"] += 1
        else:
            age_groups[">64"] += 1

    # Again we have to prepare the output in the format of the statement so we prepare it in a list
    # and then use the intro
    # Prepare the response in the requested format
    json_result = {
        "Age <18": age_groups["<18"], 
        "Age 18-29": age_groups["18-29"], 
        "Age 30-49": age_groups["30-49"], 
        "Age 50-64": age_groups["50-64"], 
        "Age >64": age_groups[">64"]
    }

    # Prepare the final JSON response
    results = {"results": [json_result]}

    # Convert the results into JSON format
    api_answer = json.dumps(results, ensure_ascii=False)

    # Return the JSON response and the status code
    output_code = 200
    return api_answer, output_code

#MILESTONE 5
#endpoint => http://localhost:8080/users_M5?countries=FR,GB,IE,IR,NO,NL,NZ,TR,US&gender=female&month=01,02,03,04,05&age=30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45
#You need to find users meeting the list of parameters in the query.
#The query should work with 1, 2, 3 or 4 of the fields combined in whatever

#countries=AU,BR,CA,CH,DE,DK,ES,FI,FR,GB,IE,IR,NO,NL,NZ,TR,US  [use the nat field for this] (if not defined consider all countries)
#gender=male,female (if not used consider all genders)
#month= 01,02,03,04,05,06,07,08,09,10,11,12 (if not used consider all the months)
#age:Any value between 18 and 99 (if not used consider all ages)

#NOTE1: IT IS NOT NEEDED TO DO CONTROL ERRORS. 
#NOTE2: You need to provide the total number of users and the 100 first users in the JOSN as follows:
#NOTE3: some times the query selection my obtain less than 100 users then print all of them

#TEXT TO DISPLAY IN CLIENT:

# NUM. USERS:165
# User 1:
# 	-Name: Linne Hoogkamer
# 	-Gender: female
# 	-Age: 40
# 	-Country: Netherlands
# 	-Postcode: 42173
# 	-Email: linne.hoogkamer@example.com
# User 2:
# 	-Name: Enola Martin
# 	-Gender: female
# 	-Age: 43
# 	-Country: France
# 	-Postcode: 93894
# 	-Email: enola.martin@example.com
# User 3:
# 	-Name: Mathilda Skjevik
# 	-Gender: female
# 	-Age: 42
# 	-Country: Norway
# 	-Postcode: 6630
# 	-Email: mathilda.skjevik@example.com
# User 4:
# 	-Name: Valentine Thomas
# 	-Gender: female
# 	-Age: 35
# 	-Country: France
# 	-Postcode: 84452
# 	-Email: valentine.thomas@example.com
#.....
#.....
#.....
#User 100:
# 	-Name: Cecilia Asphaug
# 	-Gender: female
# 	-Age: 42
# 	-Country: Norway
# 	-Postcode: 5052
# 	-Email: cecilia.asphaug@example.com

from urllib.parse import parse_qs

def users_M5(query_string):

    # First, parse the query parameters from the string using '?' to separate from our base URL
    query_string = query_string.split('?', 1)[-1]
    params = parse_qs(query_string) #parse_qs is used to parse a query string into a dictionary where the keys are parameter names and the values are lists of values for those parameters.

    # Extract 'countries', 'gender', 'month', and 'age' from the parameters and if a parameter isn't provided, set it to None
    countries = params.get('countries', [None])[0].split(',') if 'countries' in params else None
    genders = params.get('gender', [None])[0].split(',') if 'gender' in params else None
    months = params.get('month', [None])[0].split(',') if 'month' in params else None
    ages = params.get('age', [None])[0].split(',') if 'age' in params else None

    # Open JSON where data is at:
    with open('json_rand_users_5000.json', 'r', encoding='utf-8') as file:
        data = json.load(file)

    # Initialize an empty list to hold users that match the query parameters
    matching_users = []

    # Iterate through the users in the JSON data to filter based on the query parameters
    for user in data['results']:
        # Check fields, if one is not provided we skip to next user:

        if genders and user['gender'] not in genders:
            continue

        if months:
            birth_month = user['dob']['date'].split('-')[1]  # Extract the month from the date
            if birth_month not in months:
                continue

        if ages:
            age = user['dob']['age']  
            if str(age) not in ages:
                continue

        if countries and user['nat'] not in countries:
            continue
       
        # If all conditions are met, add the user to the matching list:
        matching_users.append(user)

    # Prepare the output displayed
    output_lines = [f"NUM. USERS: {len(matching_users)}"]
    
    # Loop over the first 100 matching users (limit to 100 to prevent too much output)
    
    # Iterate through the first 100 matching users
    for i, user in enumerate(matching_users[:100], start=1):
        user_data = {
            "User": i,
            "Name": f"{user['name']['first']} {user['name']['last']}",
            "Gender": user['gender'],
            "Age": user['dob']['age'],
            "Country": user['nat'],
            "Postcode": user['location']['postcode'],
            "Email": user['email']
        }
        # Append user data to the output list
        output_lines.append(user_data)
    
    # Prepare the final JSON response with user data
    results = {"results": output_lines}
    
    # Convert the results into JSON format
    api_answer = json.dumps(results, ensure_ascii=False)
    
    # Return the JSON response and the status code
    output_code = 200
    return api_answer, output_code
    
#MILESTONE 6
#We have to repeat the same exercise than in M5 but this time we need to implement control error
#FORMAT CONTROL:
# Countries: If a country is not in the available list. Generate an error and inform the user which country(s) is(are) wrong with a mesasge
# Gender: If a gender is not in the list of genders. Generate an error and inform the user which gender(s) is(are) wrong with a message
# Month: If the month is not a valid value (e.g., 15). Generate an error and inform the user which month(s) is(are) wrong with a message
# Age: if the age is not in the range 18-99 it is a wrong value. Generate an error and inform the user which age(s) is(are) wrong with a message
# Field: If after ? there is an field (e.g., extra_field) different that countries, gender, age and month report the error.
# The output message should be json file with an error per category including all the wrong values employed.
#
#NOTES:
#1- The succeed of the process should report a status code 200.
#2- The failure of the process due to an error should report the status 400 (Bad request)
#3- You need to include all the errors so you should not stop the verification of the format when you find the first error.

#TEXT TO DISPLAY IN CLIENT (it is assuming all errors are present in the created json based on the following request: )
 
#JOSN output
#{"results": [{"Country Error": "The following country IDs are not supported: IT,SW,MX", 
#"Gender Error": "The following gender options are not supported: women,men", 
#"Month Error": "The following month options are not supported: 15,23", 
#"Age Error": "The following month options are not supported: 12,15,123", 
#"Field Error": "The following fields are not valid: city"}]}

#IF THERE IS AN ERROR ONLY IN 1 or 2 FIELDS THE JSON SHOULD ONLY INCLUDE THOSE ERRORS

def users_control_error_M6(text):

    # Fristly, we define the valid range of values of each parameter
    valid_countries = ['AU', 'BR', 'CA', 'CH', 'DE', 'DK', 'ES', 'FI', 'FR', 'GB', 'IE', 'IR', 'NO', 'NL', 'NZ', 'TR', 'US']
    valid_genders = ['male', 'female']
    valid_months = [f"{i:02d}" for i in range(1, 13)]  # '01' to '12'
    valid_age_range = [str(a) for a in list(range(18, 100))]  # Ages 18 to 99
    
    # Now, parse the query parameters:
    params  = None
    param_fields = None

    # We check if there are query parameters after the '?', if there are, we add them to the dictionary params, and their name into a list called params_fields
    if '?' in text:
        query_string = text.split('?', 1)[-1]
        params = parse_qs(query_string)
        param_fields = [a for a in params]
    
    #If this list is not empty, then we remove the fields that are valid
    if param_fields:
        if 'countries' in param_fields:
            param_fields.remove('countries')
        if 'gender' in param_fields:
            param_fields.remove('gender')
        if 'age' in param_fields:
            param_fields.remove('age')
        if 'month' in param_fields:
            param_fields.remove('month')
        # We also check these fields which are necessarry afterwards in the M7
        if 'access_token' in param_fields:
            param_fields.remove('access_token')
        if 'user_ID' in param_fields:
            param_fields.remove('user_ID')
            
    # Extract 'countries', 'gender', 'month', and 'age' from the parameters and if a parameter isn't provided, set it to None as done previosuly        
    countries = params.get('countries', [None])[0].split(',') if params and 'countries' in params else valid_countries
    genders = params.get('gender', [None])[0].split(',') if params and 'gender' in params else valid_genders
    months = params.get('month', [None])[0].split(',') if params and 'month' in params else valid_months
    ages = params.get('age', [None])[0].split(',') if params and 'age' in params else valid_age_range

    # Initialize error collections
    errors = {
        "Country Error": [],
        "Gender Error": [],
        "Month Error": [],
        "Age Error": [],
        "Field Error": []
    }

    #Validate each parameter to see whether they are valid or not, if they are not, then we append the corresponding error to the dictionary 'errors'
    for country in countries:
        if country not in valid_countries:
            errors["Country Error"].append(country)

    for gender in genders:
        if gender not in valid_genders:
            errors["Gender Error"].append(gender)

    for month in months:
        if month not in valid_months:
            errors["Month Error"].append(month)

    for age in ages:
        if str(age) not in valid_age_range:
            errors["Age Error"].append(str(age))
    
    for field in param_fields:
            errors['Field Error'].append(field)


    # if the dictionary errors has stored any value then we prepare the error that is going to appear to the user:
    if any(errors.values()):
        # Filter out empty error lists
        filtered_errors = {k: v for k, v in errors.items() if v}

        c = "The following country IDs are not supported: "
        if 'Country Error' in filtered_errors.keys():
            filtered_errors['Country Error'] = c + ", ".join(filtered_errors['Country Error'])

        c = "The following age options are not supported: "
        if "Age Error" in filtered_errors.keys():
            filtered_errors["Age Error"] = c + ", ".join(filtered_errors['Age Error'])

        c = "The following gender options are not supported: "
        if "Gender Error" in filtered_errors.keys():
            filtered_errors["Gender Error"] = c + ", ".join(filtered_errors['Gender Error'])

        c = "The following month options are not supported: "
        if "Month Error" in filtered_errors.keys():
            filtered_errors['Month Error'] = c + ", ".join(filtered_errors['Month Error'])

        c = "The following field options are not supported: "
        if "Field Error" in filtered_errors.keys():
            filtered_errors['Field Error'] = c + ", ".join(filtered_errors['Field Error'])
    
        response = {"results": [filtered_errors]}

        return json.dumps(response), 400

    return users_M5(text)

#MILESTONE 7
#endpoint => http://localhost:8080/access_token_M7?access_token=$P$BZlw7PF1j.XDezr9sUj6moCjBlSx/e0&countries=FR,GB,IE,IR,NO,NL,NZ,TR,US
#&user_ID=9123&gender=male,female&month=01,05,06,12&age=30,31,32,33,34,35

#We have to repeat the same exercise than in M5 but this time we need to controll whether the userID and access token are correct.
#To this end we will use a query as follow where the access token parameter appears as another field. But it is a compulsory field.
#So we basically need to verify if the access token is there and if so we call the M6 endpoint.
#The URL will look like as follow:
#http://localhost:8080/access_token_M7?user_ID=127821&access_token=XXASJasnsna?.12123jk2amsass?&countries=FR,ES&age=18,19,20,21,22&gender=male
#You need to validate the user ID and access token in the list of valids access tokens.


#TEXT OTPIONS TO DISPLAY IN CLIENT FOR ERRORS (if access token and user ID correct then deliver answer associated to M6)

#1- No fields included (could be one of them or both of them) 
#{"results": [{"user_ID Error": "user_ID field not included in the parameters.", 
#"Access_Token Error": "Access_Token field not included in the parameters."}]}

#2- UserID does not exist
#{"results": [{"user_ID Error": "user_ID 2131 not found in the database."}]}

#3- User ID exists but access_token wrong
#{"results": [{"access_token Error": "access_token 7423hjmn23nsd.121 wrong for the userID 9123"}]}

import csv

def access_token_M7(text):

    # Read the parameters and get the values of access_token and the user_ID which we are interested in, if not set to None
    params = parse_qs( text.split('?', 1)[-1])
    user_id = params.get('user_ID', [None])[0].split(',') if params and 'user_ID' in params else None
    access_token = params.get('access_token', [None])[0].split(',') if params and 'access_token' in params else None

    # Now, read the CSV file containing stored access tokens and user IDs, and store those in a dictionary called: tokens
    tokens = {}
    with open('file_access_token_M7.csv', mode='r') as file:
        csv_reader = csv.reader(file, delimiter=';')
        for row in csv_reader:
            userid, accesstoken = row
            tokens[userid] = accesstoken

    # Check for any missing parameter
    errors = {}
    if 'user_ID' not in params.keys():
        print(params)
        errors["user_ID Error"] = "user_ID field not included in the parameters."
    if 'access_token' not in params.keys():
        errors["Access_Token Error"] = "Access_Token field not included in the parameters."

    # If either or both fields are missing, return the error(s):
    if errors:
        return json.dumps({"results": [errors]}), 400

    # If the two fields are added, then check they correspond to the right value, if not, dump an error to the user:
    for i in range(len(user_id)):
        if user_id[i] not in tokens.keys():
            return json.dumps({"results": [{"user_ID Error": f"user_ID {user_id} not found in the database."}]}), 404

        if tokens[user_id[i]] != access_token[i]:
            return json.dumps({"results": [{"access_token Error": f"access_token {access_token} wrong for the userID {user_id}"}]}), 401

    # If both fields are given and the values are alright, then check the other parameters by calling M6:
    return users_control_error_M6(text)


class MyServer(BaseHTTPRequestHandler):
    def do_GET(self):
        end_point=self.path
        
        if(end_point.startswith("/test_connectivity_M1")):
            api_answer,output_code= test_connectivity_M1(end_point)
            
        elif(end_point.startswith("/count_countries_M2")):
            api_answer,output_code= country_count_M2(end_point)

        
        elif(end_point.startswith("/count_countries_M3")):
            api_answer,output_code= country_count_M3(end_point)

            
        elif(end_point.startswith("/age_groups_M4")):
            api_answer,output_code= age_groups_M4(end_point)

            
        elif(end_point.startswith("/users_M5")):
            api_answer,output_code= users_M5(end_point)

        elif(end_point.startswith("/users_control_error_M6")):
            api_answer,output_code= users_control_error_M6(end_point)
        
        elif(end_point.startswith("/access_token_M7")):
            api_answer,output_code= access_token_M7(end_point)

        
        else:
            results = {}
            json_error={}
            array = []             
            json_error['Endpoint Error']="The provided endpoint does not exist."
            array.append(json_error)
            results['results'] = array
            api_answer=json.dumps(results, ensure_ascii=False)
            output_code=400
        
        self.send_response(output_code)
        self.send_header("Content-type", "text/json")
        self.end_headers() 
        self.wfile.write(bytes(api_answer, "utf-8"))
        
if __name__ == "__main__":        
    webServer = HTTPServer((hostName, serverPort), MyServer)
    print("Server started http://%s:%s" % (hostName, serverPort))

    try:
        webServer.serve_forever()
    except KeyboardInterrupt:
        pass

    webServer.server_close()
    print("Server stopped.")

Server started http://localhost:8080


127.0.0.1 - - [23/Oct/2024 12:24:32] "GET /count_countries_M3?countries=DK,ES,GB,IE,IR,NO,NL,NZ HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2024 12:24:34] "GET /count_countries_M2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2024 12:24:46] "GET /age_groups_M4 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2024 12:24:48] "GET /count_countries_M2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2024 12:25:28] "GET /count_countries_M2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Oct/2024 12:25:28] "GET /favicon.ico HTTP/1.1" 400 -
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 57453)
Traceback (most recent call last):
  File "C:\Users\Marina\anaconda3\Lib\socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "C:\Users\Marina\anaconda3\Lib\socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "C:\Users\Marina\anaconda3\Lib\socketserver.py", line 362, in finish_request